<h2>Note:</h2>

This notebook contains optional part that is not related to Mozgalo competition.

Notebook applies our trained model on a harder dataset with cca 30000 images and roughly 256 meaningful categories of objects.

In [1]:
import sklearn
from sklearn import decomposition
import torchfile
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import os
from shutil import copyfile
import binascii
from sklearn.mixture import GaussianMixture
import random
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
import warnings
warnings.filterwarnings('ignore')

In [2]:
# since dataset is much bigger it needed to be split in chunks
dataset_chunk_0 = torchfile.load('256_set/featuresChunk0.t7')
dataset_chunk_1 = torchfile.load('256_set/featuresChunk1.t7')
dataset_chunk_2 = torchfile.load('256_set/featuresChunk2.t7')
dataset_chunk_3 = torchfile.load('256_set/featuresChunk3.t7')
dataset_chunk_4 = torchfile.load('256_set/featuresChunk4.t7')
dataset_chunk_5 = torchfile.load('256_set/featuresChunk5.t7')
dataset_chunk_6 = torchfile.load('256_set/featuresChunk6.t7')

In [3]:
dataset_list = []
dataset_list.append(dataset_chunk_0)
dataset_list.append(dataset_chunk_1)
dataset_list.append(dataset_chunk_2)
dataset_list.append(dataset_chunk_3)
dataset_list.append(dataset_chunk_4)
dataset_list.append(dataset_chunk_5)
dataset_list.append(dataset_chunk_6)

In [4]:
# chunk gluing
dataset = {}

dataset['image_list'] = dataset_list[0]['image_list']
dataset['features'] = dataset_list[0]['features']


for i in range(1, len(dataset_list)):
    dataset['image_list']+=dataset_list[i]['image_list']
    dataset['features'] = np.concatenate((dataset['features'], dataset_list[i]['features']))

In [5]:
print (len(dataset['features']))

30607


In [6]:
pca = decomposition.PCA(n_components=1000, svd_solver='full')
pca.fit(dataset['features'])

PCA(copy=True, iterated_power='auto', n_components=1000, random_state=None,
  svd_solver='full', tol=0.0, whiten=False)

In [14]:
transformed_data = pca.transform(dataset['features'])

In [ ]:
import sklearn.cluster

transformed_data_unshuffled = pca.transform(dataset['features'])

n_clusters = 256
        
clusterer = sklearn.cluster.AgglomerativeClustering(n_clusters)
clusterer.fit(transformed_data_unshuffled)

In [ ]:
print clusterer.labels_

In [ ]:
harder_dataset_dir_path = './256_dataset/'

if not os.path.exists(harder_dataset_dir_path):
        os.makedirs(harder_dataset_dir_path)
        
for i in range(n_clusters):
    newpath = harder_dataset_dir_path + 'class_' + str(i)
    if not os.path.exists(newpath):
        os.makedirs(newpath)

for i in range(len(dataset['image_list'])):
    if i % 1000 == 0:
        print i
    path = '../' + dataset['image_list'][i]
    
    #feature_vec = transformed_data_unshuffled[i]
    #class_val = clusterer.predict(feature_vec)
    save_path = harder_dataset_dir_path + r'class_' + str(clusterer.labels_[i])
    tokens = path.decode("utf-8").split('/')
    copyfile(path.decode("utf-8"), save_path + '/' + tokens[len(tokens) - 1])

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000


In [55]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
import numpy as np

In [68]:
Z = linkage(transformed_data[:500], 'ward')
labels = dataset['image_list']

In [72]:
plt.figure(figsize=(50, 300))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    Z,
    leaf_rotation=0.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
    orientation='right',
    labels=labels
)
plt.show()

In [16]:
plt.savefig('foo.png', bbox_inches='tight')

In [47]:


# Load required modules
import pandas as pd 
import scipy.spatial
import scipy.cluster
import json
import matplotlib.pyplot as plt

# Determine distances (default is Euclidean)
dataMatrix = transformed_data[:100]
distMat = scipy.spatial.distance.pdist( dataMatrix )

# Cluster hierarchicaly using scipy
clusters = scipy.cluster.hierarchy.linkage(distMat, method='ward')
T = scipy.cluster.hierarchy.to_tree( clusters , rd=False )



In [48]:
# Create dictionary for labeling nodes by their IDs
labels = dataset['image_list']
id2name = dict(zip(range(len(labels)), labels))

# Draw dendrogram using matplotlib to scipy-dendrogram.pdf
#scipy.cluster.hierarchy.dendrogram(clusters, labels=labels, orientation='right')
#plt.savefig("scipy-dendrogram.png")

# Create a nested dictionary from the ClusterNode's returned by SciPy
def add_node(node, parent, cntr=0 ):
    # First create the new node and append it to its parent's children
    newNode = dict( node_id=node.id, children=[] )
    parent["children"].append( newNode )
    
    # Recursively add the current node's children
    if node.left: 
        add_node( node.left, newNode, cntr+1 )
    if node.right: 
        add_node( node.right, newNode, cntr+1 )

# Initialize nested dictionary for d3, then recursively iterate through tree
d3Dendro = dict(children=[], name="Root1")
add_node( T, d3Dendro )

# Label each node with the names of each leaf in its subtree
def label_tree( n ):
    # If the node is a leaf, then we have its name
    if len(n["children"]) == 0:
        leafNames = [ id2name[n["node_id"]] ]
    
    # If not, flatten all the leaves in the node's subtree
    else:
        leafNames = reduce(lambda ls, c: ls + label_tree(c), n["children"], [])

    # Delete the node id since we don't need it anymore and
    # it makes for cleaner JSON
    del n["node_id"]

    # Labeling convention: "-"-separated leaf names
    n["name"] = name = "-".join(sorted(map(str, leafNames)))
    
    return leafNames

label_tree( d3Dendro["children"][0] )

# Output to JSON
json.dump(d3Dendro, open("d3-dendrogram.json", "w"), sort_keys=True, indent=4)